# 🤖 Natural Language to SQL Agent - Testing & Validation

## 📋 Overview
This notebook provides a comprehensive testing environment for the NL2SQL (Natural Language to SQL) agent. It demonstrates the agent's ability to understand natural language questions and convert them into SQL queries.

## 🎯 What You'll Learn
- Setting up and configuring the SQL agent
- Testing database connectivity
- Streaming vs non-streaming responses
- Vector search integration for intelligent table discovery
- Debugging agent behavior
- Comparing different LLM providers

## 🏗️ Architecture
```
User Question → Agent → Tool Selection → SQL Execution → Formatted Answer
                 ↓
            [Vector Search]
            [List Tables]
            [Get Schema]
            [Execute Query]
```

## 📝 Prerequisites
- Database connection configured in `src/config/db_schema.py`
- LLM API keys set (OpenAI or Ollama)
- Python environment with all dependencies installed

---

## 🔧 Step 1: Setup and Imports

Configure the Python environment and import required modules.

**What This Does**:
- Adds project root to Python path for module imports
- Imports LLM configuration utilities
- Imports agent creation functions
- Imports database tools manager

**Expected Output**: ✅ Imports successful

In [1]:
import sys
import os

# Add project root to path
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.config.models import default_llm, default_embedding_model
from src.agents.agent import get_agent
from src.agents.tools import db_tool_manager
from src.config.models import get_llm

print("✅ Imports successful")

c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ Imports successful


c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'DatabaseUser'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Event'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Schema'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Object'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain

---

## ⚙️ Step 2: Configure LLM Provider

Choose which language model provider to use for the agent.

### Available Options:
- **OpenAI** (gpt-4o-mini, gpt-4, gpt-3.5-turbo)
  - Pros: High quality, reliable, fast
  - Cons: Requires API key, costs per token
  
- **Ollama** (llama3, mistral, etc.)
  - Pros: Free, runs locally, privacy-focused
  - Cons: Requires local installation, slower

**Current Selection**: This cell shows which provider is active.

**To Switch**: Uncomment the desired provider and comment out the other.

In [ ]:
# Option 1: Use Ollama
default_llm["provider"] = "ollama"
default_llm["model"] = "llama3-groq-tool-use"

# Option 2: Use OpenAI (uncomment to use)
# default_llm["provider"] = "openai"
# default_llm["model"] = "gpt-4o-mini"


print(f"Using: {default_llm['provider']}:{default_llm['model']}")
print(f"Embedding: {default_embedding_model['provider']}:{default_embedding_model['model']}")

Using: openai:gpt-4o-mini
Embedding: openai:text-embedding-3-small


---

## 🤖 Step 4: Initialize SQL Agent

Create the agent instance with configured LLM and database tools.


In [3]:
# Get the agent
agent = get_agent(default_llm["provider"], default_llm["model"])
print("✅ Agent initialized successfully")

✅ Agent initialized successfully


---

## 🔍 Step 4.5: Verify Agent Configuration

Inspect the agent's tools and configuration to ensure everything is properly set up.

**What This Shows**:
- Complete list of available tools
- Tool descriptions
- Total tool count
- Agent cache status

**💡 Important Note**: 
If you modify the system prompt in `src/config/prompt.py`, you need to **restart the kernel** to clear the agent cache, otherwise the old prompt will still be used.

In [4]:
# Verify the agent has the right tools and prompt
llm = get_llm()
tools = db_tool_manager.get_tools(llm=llm)
print("Agent Tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description[:80]}...")

print(f"\nTotal tools: {len(tools)}")

# Check if agent is cached or needs refresh
from src.agents.agent import _agents_cache
print(f"\nCached agents: {list(_agents_cache.keys())}")
print("\n⚠️ Note: If you changed the prompt, clear cache by restarting kernel")

Agent Tools:
  - sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from ...
  - sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and...
  - sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the data...
  - sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Alwa...
  - list_all_tables: List ALL tables from ALL schemas in the database.
Returns fully qualified table ...
  - get_table_schema: Get detailed schema for a specific table including columns, types, and keys.

Ar...
  - get_database_summary: Get high-level database overview: total tables, schemas, and sample tables....
  - find_relevant_tables: Search for relevant database tables based on a natural language question.
Use th...
  - search_tables_by_keyword: Search for tables whose names or schemas contain specific keywords.
Useful for f...

Total tools: 9

Cached 

---

## 🎬 Step 5: Test Streaming Response

Test the agent with a real question and observe its thinking process in real-time.

**Streaming Benefits**:
- See agent's reasoning step-by-step
- Real-time feedback (better UX)
- View tool selections as they happen
- Observe plan updates

**What You'll See**:
- 📋 Plan updates (agent's strategy)
- 🔧 Tool calls and arguments
- 💬 Final formatted answer
- ✅ Success confirmation with chunk count

**Test Question**: "List the top 5 customers by order count"
- Requires finding relevant tables
- Needs SQL with ORDER BY and LIMIT
- Should format results nicely

In [ ]:
from src.agents.agent import stream_agent
from src.ui.utils import extract_content_from_chunk, extract_todos

question = "List the top 5 customers by order count"
config = {"configurable": {"thread_id": "test-stream-1"}}

print(f"Question: {question}\n")
print("="*80)

full_response = ""
current_plan = ""
chunk_count = 0

try:
    for chunk in stream_agent(agent, question, config):
        chunk_count += 1
        
        # Extract plan updates
        new_plan = extract_todos(chunk)
        if new_plan and new_plan != current_plan:
            current_plan = new_plan
            print(f"\n📋 Plan Update (chunk {chunk_count}):")
            print(current_plan)
            print("-" * 40)
        
        # Extract content
        new_contents = extract_content_from_chunk(chunk)
        for content in new_contents:
            full_response += content
            print(content, end="", flush=True)
    
    print("\n" + "="*80)
    print(f"\n✅ Processed {chunk_count} chunks")
    
    if current_plan:
        print("\n📋 Final Plan:")
        print(current_plan)
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

Question: List the top 5 customers by order count



INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Searching schema vector store for question: top customers by order count


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[tools] --- Relevant Table 1: Sales.SalesOrderHeader ---
Table: Sales.SalesOrderHeader

    Schema Details:
    Table: Sales.SalesOrderHeader

Columns:
  - SalesOrderID: INTEGER NOT NULL
  - RevisionNumber: TINYINT NOT NULL
  - OrderDate: DATETIME NOT NULL
  - DueDate: DATETIME NOT NULL
  - ShipDate: DATETIME NULL
  - Status: TINYINT NOT NULL
  - OnlineOrderFlag: NULL NOT NULL
  - SalesOrderNumber: NVARCHAR(25) COLLATE "SQL_Latin1_General_CP1_CI_AS" NOT NULL
  - PurchaseOrderNumber: NULL NULL
  - AccountNumber: NULL NULL
  - CustomerID: INTEGER NOT NULL
  - SalesPersonID: INTEGER NULL
  - TerritoryID: INTEGER NULL
  - BillToAddressID: INTEGER NOT NULL
  - ShipToAddressID: INTEGER NOT NULL
  - ShipMethodID: INTEGER NOT NULL
  - CreditCardID: INTEGER NULL
  - CreditCardApprovalCode: VARCHAR(15) COLLATE "SQL_Latin1_General_CP1_CI_AS" NULL
  - CurrencyRateID: INTEGER NULL
  - SubTotal: MONEY NOT NULL
  - TaxAmt: MONEY NOT NULL
  - Freight: MONEY NOT NULL
  - TotalDue: MONEY NOT NULL
  - Co

...

[tools] ```sql
SELECT TOP 5 CustomerID, COUNT(SalesOrderID) AS OrderCount 
FROM Sales.SalesOrderHeader 
GROUP BY CustomerID 
ORDER BY OrderCount DESC;
```

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[tools] [(11091, 28), (11176, 28), (11185, 27), (11200, 27), (11223, 27)]

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[model] **Ready for execution** ✅ **Query executed successfully:**
Customer: 11091 | Orders: 28  
Customer: 11176 | Orders: 28  
Customer: 11185 | Orders: 27  
Customer: 11200 | Orders: 27  
Customer: 11223 | Orders: 27  

**Reasoning:**
- **Tables**: The query was executed on the `Sales.SalesOrderHeader` table, which contains order information including customer IDs.
- **Columns**: The `CustomerID` column identifies the customer, and `SalesOrderID` is counted to determine the number of orders per customer.
- **Logic**: The query groups the results by `CustomerID`, counts the number of orders for each customer, and sorts the results in descending order to get the top 5 customers by order count.
- **SQL**: 

```sql
SELECT TOP 5 CustomerID, COUNT(SalesOrderID) AS OrderCount 
FROM Sales.SalesOrderHeader 
GROUP BY CustomerID 
ORDER BY OrderCount DESC;
```

✅ Processed 369 chunks


---

## 📊 Step 6: Test Non-Streaming Query

Test a different question using non-streaming mode (simpler, waits for complete answer).

**Non-Streaming Mode**:
- Waits for complete response before returning
- No intermediate output
- Simpler code
- Good for batch processing

**Test Question**: "What is the total sales amount by year?"
- Requires aggregation (SUM)
- Needs GROUP BY clause
- Tests date/time handling

**Use Case**: When you don't need real-time updates and prefer a single complete response.

In [ ]:
question = "What is the total sales amount by year?"

print(f"Question: {question}\n")
print("="*80)

# Include thread_id in config for checkpointer
config = {"configurable": {"thread_id": "test-session-2"}}
response = agent.invoke({"input": question}, config=config)

print("\n" + "="*80)
print("\nFinal Answer:")
print(response.get("output", "No output"))

---

## 🔗 Step 7: Test Complex Query with Joins

Test the agent's ability to handle multi-table queries requiring JOINs.


In [ ]:
question = "Show me the top 5 products by revenue with their category names"

print(f"Question: {question}\n")
print("="*80)

# Include thread_id in config for checkpointer
config = {"configurable": {"thread_id": "test-session-3"}}
response = agent.invoke({"input": question}, config=config)

print("\n" + "="*80)
print("\nFinal Answer:")
print(response.get("output", "No output"))



---

## 🐛 Step 10: Debug Mode - View Agent's Thinking Process

Enable verbose/debug mode to see detailed agent internals and reasoning steps.

### What Verbose Mode Shows:
- 🔍 **Tool Selections**: Which tools the agent chooses
- 📝 **Tool Arguments**: Parameters passed to each tool
- 💭 **Reasoning**: Agent's thought process between steps
- 🔄 **Intermediate States**: Variables and context updates
- ⚠️ **Errors**: Detailed error messages and stack traces

### When to Use Debug Mode:
- ✅ Agent giving wrong answers
- ✅ Understanding tool selection logic
- ✅ Debugging prompt engineering
- ✅ Learning agent behavior
- ✅ Troubleshooting errors

### Test Question:
"How many employees are in each department?"

**Expected Flow**:
1. Find relevant tables (Employee, Department)
2. Get schema for those tables
3. Generate GROUP BY query
4. Execute query
5. Format results

### 💡 Pro Tip:
Compare the output with non-verbose mode to understand what's hidden by default.

**⚠️ Note**: Debug mode produces **a lot** of output. Use sparingly!

In [ ]:
# Enable verbose mode to see agent's thinking process
from langchain_core.globals import set_verbose, set_debug

set_verbose(True)
set_debug(True)

question = "How many employees are in each department?"
config = {"configurable": {"thread_id": "test-session-4"}}

print(f"Question: {question}\n")
print("="*80)

full_response = ""
current_plan = ""
chunk_count = 0

try:
    for chunk in stream_agent(agent, question, config):
        chunk_count += 1
        
        # Extract plan updates
        new_plan = extract_todos(chunk)
        if new_plan and new_plan != current_plan:
            current_plan = new_plan
            print(f"\n📋 Plan Update (chunk {chunk_count}):")
            print(current_plan)
            print("-" * 40)
        
        # Extract content
        new_contents = extract_content_from_chunk(chunk)
        for content in new_contents:
            full_response += content
            print(content, end="", flush=True)
    
    print("\n" + "="*80)
    print(f"\n✅ Processed {chunk_count} chunks")
    
    if current_plan:
        print("\n📋 Final Plan:")
        print(current_plan)
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Disable verbose mode
    set_verbose(False)
    set_debug(False)

Question: How many employees are in each department?

[chain/start] [chain:LangGraph] Entering Chain run with input:
{
  "messages": [
    [
      "user",
      "How many employees are in each department?"
    ]
  ]
}
[chain/start] [chain:LangGraph > chain:model] Entering Chain run with input:
[inputs]
[llm/start] [chain:LangGraph > chain:model > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: \nYou are a senior database professional who specializes in SQL and excels at:\n- Understanding complex relational schemas\n- Explaining schema semantics clearly\n- Translating natural language requests into precise, optimized SQL\n\nYou have access to these tools for understanding and working with the database:\n\n**SCHEMA DISCOVERY TOOLS (ALWAYS START HERE):**\n1. find_relevant_tables(question) - **USE THIS FIRST** - Intelligent semantic search for relevant tables based on natural language\n2. search_tables_by_keyword(keyword) - Search tables by name/keyword pattern m

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[llm/end] [chain:LangGraph > chain:model > llm:ChatOpenAI] [1.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls",
          "model_name": "gpt-4o-mini-2024-07-18",
          "system_fingerprint": "fp_29330a9688",
          "service_tier": "default"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "response_metadata": {
              "finish_reason": "tool_calls",
              "model_name": "gpt-4o-mini-2024-07-18",
              "system_fingerprint": "fp_29330a9688",
              "service_tier": "default",
              "model_provider": "openai"
            },
            "type": "ai",
            "id": "lc_run--019ba262-384a-

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[tool/end] [chain:LangGraph > chain:tools > tool:find_relevant_tables] [1.00s] Exiting Tool run with output:
"content="--- Relevant Table 1: HumanResources.Department ---\nTable: HumanResources.Department\n\nSchema Details:\nTable: HumanResources.Department\n\nColumns:\n  - DepartmentID: SMALLINT NOT NULL\n  - Name: NULL NOT NULL\n  - GroupName: NULL NOT NULL\n  - ModifiedDate: DATETIME NOT NULL\n\nPrimary Key: DepartmentID\n\n\n\nSample Data (first 3 rows):\n[(1, 'Engineering', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0)), (2, 'Tool Design', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0)), (3, 'Sales', 'Sales and Marketing', datetime.datetime(2008, 4, 30, 0, 0))]\n\nThis table can be queried to answer questions about HumanResources.Department.\n\n--- Relevant Table 2: HumanResources.Department ---\nTable: HumanResources.Department\n\nSchema Details:\nTable: HumanResources.Department\n\nColumns:\n  - DepartmentID: SMALLINT NOT NULL\n  - Name: NULL

...

[llm/end] [chain:LangGraph > chain:tools > tool:sql_db_query_checker > chain:LLMChain > llm:ChatOpenAI] [1.73s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```sql\nSELECT d.Name AS Department, COUNT(ed.BusinessEntityID) AS EmployeeCount \nFROM HumanResources.Department d \nLEFT JOIN HumanResources.EmployeeDepartmentHistory ed ON d.DepartmentID = ed.DepartmentID \nGROUP BY d.Name;\n```",
        "generation_info": {
          "finish_reason": "stop",
          "model_name": "gpt-4o-mini-2024-07-18",
          "system_fingerprint": "fp_29330a9688",
          "service_tier": "default"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "```sql\nSELECT d.Name AS Department, COUNT(ed.BusinessEntityID) AS EmployeeCount \nF

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[llm/end] [chain:LangGraph > chain:model > llm:ChatOpenAI] [2.56s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls",
          "model_name": "gpt-4o-mini-2024-07-18",
          "system_fingerprint": "fp_29330a9688",
          "service_tier": "default"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "response_metadata": {
              "finish_reason": "tool_calls",
              "model_name": "gpt-4o-mini-2024-07-18",
              "system_fingerprint": "fp_29330a9688",
              "service_tier": "default",
              "model_provider": "openai"
            },
            "type": "ai",
            "id": "lc_run--019ba262-5094-

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[llm/end] [chain:LangGraph > chain:model > llm:ChatOpenAI] [10.08s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "**Ready for execution** ✅ **Query executed successfully:**\nDepartment: Engineering | Employees: 7  \nDepartment: Tool Design | Employees: 4  \nDepartment: Sales | Employees: 18  \nDepartment: Marketing | Employees: 10  \nDepartment: Purchasing | Employees: 13  \nDepartment: Research and Development | Employees: 4  \nDepartment: Production | Employees: 180  \nDepartment: Production Control | Employees: 6  \nDepartment: Human Resources | Employees: 6  \nDepartment: Finance | Employees: 11  \nDepartment: Information Services | Employees: 10  \nDepartment: Document Control | Employees: 5  \nDepartment: Quality Assurance | Employees: 7  \nDepartment: Facilities and Maintenance | Employees: 7  \nDepartment: Shipping and Receiving | Employees: 6  \nDepartment: Executive | Employees: 2  \n\n**Reasoning:**\n- **Tables**: The query utilized the `Hu